In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch.utils.tensorboard import SummaryWriter

def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.constant_(m.bias, 0)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
     # 是否为批归一化层
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


In [14]:
# 定义神经网络模型
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, output_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, hidden_size)
        self.fc6 = nn.Linear(hidden_size, hidden_size)
        self.fc7 = nn.Linear(hidden_size, hidden_size)
        self.fc8 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        """ x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x)) """
        x = self.fc1(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
input_size = 2
hidden_size = 128
output_size = 4
data = torch.tensor(np.load(r"data/points_results.npy")).float().to(device)
data = data[~torch.isnan(data).any(1)]

In [16]:
data = data[0:10000,:]
print(data)

tensor([[ 2.5000, -4.3301,  4.5915,  ...,  0.4791,  0.0000,  0.8777],
        [ 2.5091, -4.3249,  4.5913,  ...,  0.4811,  0.0000,  0.8766],
        [ 2.5181, -4.3196,  4.5910,  ...,  0.4831,  0.0000,  0.8755],
        ...,
        [ 2.6438, -4.2439,  4.5877,  ...,  0.5109,  0.0000,  0.8596],
        [ 2.6527, -4.2383,  4.5875,  ...,  0.5128,  0.0000,  0.8585],
        [ 2.6616, -4.2327,  4.5873,  ...,  0.5148,  0.0000,  0.8573]],
       device='cuda:0')


In [19]:
model = NeuralNetwork(input_size, hidden_size, output_size).to(device)
model.apply(weight_init)
#torch中的apply函数通过可以不断遍历model的各个模块，并将weight_init函数应用在这些Module上
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.9)

# 假设你已经有一个名为data的数据集，包含1000000个样本
# data应该是一个Tensor，每行是一个样本，前两列是输入，后四列是输出
# 使用DataLoader来加载数据集
# DataLoader的batch_size可以根据你的实际情况进行调整
input = data[:,]
data_loader = DataLoader(TensorDataset(data[:, :2], data[:, 3:]), batch_size=32, shuffle=True)

# 训练模型
num_epochs = 10000
writer = SummaryWriter()
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        # 前向传播
        # Q1 output 不是单位向量
        # Q2 1e6个点太多了，所以nan
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar('Loss/train',loss,epoch)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 保存模型
torch.save(model.state_dict(), 'dune_model/model.pth')


Epoch [1/10000], Loss: 3.6508
Epoch [2/10000], Loss: 2.7472
Epoch [3/10000], Loss: 3.1072
Epoch [4/10000], Loss: 1.8168
Epoch [5/10000], Loss: 2.2040
Epoch [6/10000], Loss: 2.2043
Epoch [7/10000], Loss: 1.7075
Epoch [8/10000], Loss: 1.7036
Epoch [9/10000], Loss: 0.9314
Epoch [10/10000], Loss: 1.0024
Epoch [11/10000], Loss: 1.0340
Epoch [12/10000], Loss: 0.8749
Epoch [13/10000], Loss: 0.6600
Epoch [14/10000], Loss: 0.6272
Epoch [15/10000], Loss: 0.6683
Epoch [16/10000], Loss: 0.4914
Epoch [17/10000], Loss: 0.3599
Epoch [18/10000], Loss: 0.3589
Epoch [19/10000], Loss: 0.2372
Epoch [20/10000], Loss: 0.2036
Epoch [21/10000], Loss: 0.1666
Epoch [22/10000], Loss: 0.1483
Epoch [23/10000], Loss: 0.1033
Epoch [24/10000], Loss: 0.0954
Epoch [25/10000], Loss: 0.0822
Epoch [26/10000], Loss: 0.0722
Epoch [27/10000], Loss: 0.0802
Epoch [28/10000], Loss: 0.0629
Epoch [29/10000], Loss: 0.0488
Epoch [30/10000], Loss: 0.0500
Epoch [31/10000], Loss: 0.0430
Epoch [32/10000], Loss: 0.0454
Epoch [33/10000],

KeyboardInterrupt: 